In [1]:
!pip install keras==3.0.0 --upgrade --quiet

import os
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras
print(keras.__version__)

3.0.0


In [2]:
from keras import backend as K

print(K.backend())

torch


## An LSTM Autoencoder

In [3]:
# Hyperparameters
n_in = 9
# learning_rate = 1e-3
num_epoch = 300

In [4]:
class LSTMAE(keras.Model):
    def __init__(self):
        super().__init__()
        self.lstm1 = keras.layers.LSTM(100, activation='relu',
                                       input_shape=(n_in,1), return_sequences=True)
        # Can we only use return_sequence in the last LSTM encoder layer
        # and don’t use RepeatVector before the first LSTM decoder layer?

        # Maybe we will still use RepeatVector because we are actually
        # encoding the very vector.

        # self.repeatvector = keras.layers.RepeatVector(n_in)
        self.lstm2 = keras.layers.LSTM(100, activation='relu',
                                       return_sequences=True)
        self.compress = keras.layers.TimeDistributed(keras.layers.Dense(1))

    def encode(self, x):
        # return self.repeatvector(self.lstm1(x))
        return self.lstm1(x)

    def decode(self, z):
        return self.compress(self.lstm2(z))

    def call(self, x, training=False):
        return self.decode(self.encode(x))

In [5]:
from numpy import array

sequence = array([0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9])
sequence = sequence.reshape((1, n_in, 1))

model = LSTMAE()
# optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

model.compile(optimizer='adam', loss='mse')

/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:205: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [6]:
model.fit(sequence, sequence, epochs=num_epoch, verbose=0)

In [7]:
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[0.11402848 0.20080328 0.29531628 0.39493078 0.49756372 0.60109377
 0.70345855 0.8027534  0.8971434 ]


# An LSTM Autoencoder with maksing

In [8]:
class MLSTMAE(keras.Model):
    def __init__(self):
        super().__init__()
        self.lstm1 = keras.layers.LSTM(100, activation='relu',
                                       input_shape=(n_in,1))
        # The vector we repeat here is the encoding
        self.repeatvector = keras.layers.RepeatVector(n_in)
        self.lstm2 = keras.layers.LSTM(100, activation='relu',
                                       return_sequences=True)
        self.compress = keras.layers.TimeDistributed(keras.layers.Dense(1))

    def encode(self, x):
        x = keras.layers.Masking(mask_value=0.0)(x)
        return self.repeatvector(self.lstm1(x))

    def decode(self, z):
        return self.compress(self.lstm2(z))

    def call(self, x, training=False):
        return self.decode(self.encode(x))

In [9]:
model = MLSTMAE()
model.compile(optimizer='adam', loss='mse')
model.fit(sequence, sequence, epochs=num_epoch, verbose=0)

In [10]:
# demonstrate recreation
yhat = model.predict(sequence, verbose=0)
print(yhat[0,:,0])

[0.10634747 0.20153731 0.2991102  0.39874396 0.49926123 0.6000028
 0.70046484 0.80029273 0.8992775 ]
